In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset, DatasetDict
from fine_tune_util import EvalSampleDatasetTrainer, compute_metrics, preprocess_logits_for_metrics_mlm, token_length_histogram, save_dicts_to_csv, save_metrics
from datetime import datetime

/home/bhx5gh/Documents/NLP/NLP_Final_Political_Bias_Shifts/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# This should be the only part of this code that is getting modified                      #
model_name = "FacebookAI_roberta-large" 
model_path = f"../Local Models/{model_name}"                                              #                                                      
                                                                                          #
from_pretrained_params_dict = {                                                           #
    "pretrained_model_name_or_path" : model_path,                                                           #
    "device_map":"cuda:0",                                                                  #
    #"torch_dtype": torch.float16                                                          #
}                                                                                         #
                                                                                          #
'''lora_config_params_dict = {                                                               #
    "lora_alpha":1024,                                                                      #
    "lora_dropout":0.1,                                                                   #
    "r":512,                                                                               #
    "bias":"none",                                                                        #
    "task_type":TaskType.,                                                       #
}     '''                                                                                    #
                                                                                          #
quantization_params_dict = { }                                                                                         #
                                                                                          #
tokenizer_params_dict = {                                                                 #
  "truncation":True,
  "padding": True,
  "max_length":384                                                                       #
}                                                                                         #
                                                                                          #
cur_datetime = datetime.now().strftime("%Y-%m-%d %H-%M-%S")                               #
checkpoint_dir = f"../../fine_tuned_llms/{model_name}/checkpoints/{cur_datetime}"        #
metrics_dir = f"{checkpoint_dir}/metrics.json"                                              #                                                                                          
                                                                                          #
training_args_dict = {                                                                    #                 
  "output_dir":checkpoint_dir,                                                            #                              
  "per_device_train_batch_size":96, # using A100 gpu, not sure if rivanna can handle more,  #                                                             
  "per_device_eval_batch_size":96,                                                         #                                 
  "num_train_epochs":2,                
  "evaluation_strategy":"steps",                                                          #                                
  "save_strategy":"best",                                                                #                          
  "eval_steps":150,                                                                        #                  
  "save_steps":150,                                                                     #                                
  "metric_for_best_model":"perplexity",                                                                                       
  "greater_is_better":False,              # Lower perplexity is better                    #                                                                      
  "logging_dir":metrics_dir,   
  "logging_strategy": "steps" , 
  "logging_steps": 150,                                                         #                            
  "fp16":True, 
  "learning_rate": 1e-4,    
  "lr_scheduler_type":'constant',
  "eval_on_start": True,     
  "save_safetensors":False,
  "warmup_steps": 100,
  #"ddp_backend": "nccl",     
 # "torch_compile":True,                                                                 #              
  #save_total_limit=3, # only keeping best 3                                              #                                            
}                                                                                         # 
###########################################################################################

In [3]:
#load dataset

seed = 210

data = pd.read_csv("../../data/Cleaned Data/CNN_comments_clean.csv")  
comments = data["comment"].astype(str).sample(frac=0.1, random_state=seed)

train_comments, test_comments = train_test_split(comments, test_size=0.3, random_state=seed)
val_comments, test_comments = train_test_split(test_comments, test_size=0.5, random_state=seed)

train_dataset = Dataset.from_pandas(pd.DataFrame({"text": train_comments}))
val_dataset = Dataset.from_pandas(pd.DataFrame({"text": val_comments}))
test_dataset = Dataset.from_pandas(pd.DataFrame({"text": test_comments}))

dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})



In [4]:
# set up model
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast = True)
model = AutoModelForMaskedLM.from_pretrained(**from_pretrained_params_dict)
#peft_config =  LoraConfig(**lora_config_params_dict)
#model = get_peft_model(model, peft_config)
#print(model)

In [5]:
total_params = sum(p.numel() for p in model.parameters())  # Total number of parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)  # Trainable parameters

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

Total parameters: 355412057
Trainable parameters: 355412057


In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], **tokenizer_params_dict) # we don't do padding here, we let the data collater handle it

In [7]:
# Tokenize each split and remove the 'text' column
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Add 'labels' field for language modeling
tokenized_datasets = tokenized_datasets.remove_columns(["__index_level_0__"])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability = .15)

Map: 100%|██████████| 24580/24580 [00:11<00:00, 2219.08 examples/s]


In [8]:
# set up trainer
training_args = TrainingArguments(**training_args_dict)
trainer = EvalSampleDatasetTrainer(
    eval_sample_size_proportion = .25,
    seed = seed,
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics_mlm,
)

/home/bhx5gh/Documents/NLP/NLP_Final_Political_Bias_Shifts/.venv/lib64/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [9]:
trainer.train()

Step,Training Loss,Validation Loss,Perplexity
0,No log,1.776225,5.907522
10,2.076200,1.984788,7.277503
20,2.084400,1.931438,6.899417
30,2.092200,1.873838,6.513244


In [ ]:
save_metrics(trainer.state.log_history, f'{checkpoint_dir}/metrics.json')

hyperparams = {
    "from_pretrained_params": from_pretrained_params_dict,
    #"lora_config_params":lora_config_params_dict,
    "quantization_params":quantization_params_dict,
    "tokenizer_params":tokenizer_params_dict,
    "training_args":training_args_dict
}


save_dicts_to_csv(hyperparams, '/home/bhx5gh/Documents/NLP/NLP_Final_Political_Bias_Shifts/fine_tuned_llms/mistralai_Mistral-7B-v0_1/runs.csv',
model_name, cur_datetime)
